In [2]:
!pip install geopandas

In [3]:
!pip install imageio

In [4]:
!pip install laspy

In [5]:
#!pip install rasterio

In [6]:
!pip install folium

In [7]:
import json
import urllib.request as urllib
import pickle

In [8]:
pre = 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/'
trail ='/ept.json'

In [ ]:
gdf_dict ={'name':[], 'year':[], 'path':[], 'geometry':[], 'bounds':[]}
counter = 0
with open('../data/filename.txt') as f:
    for line in f:
        line_ = line.strip('/\n')
        gdf_dict['name'].append(line_)
        year = line_.split('_')[-1]
        gdf_dict['year'].append(year)
        path = pre + line_ + trail
        gdf_dict['path'].append(path)
        try:
            j_res = json.load(urllib.urlopen(path))
            wkt = j_res['srs']['wkt']
            bounds = j_res['bounds']
        except:
            print('path: {}\ncounter: {}'.format(path, counter))
            wkt = None
            bounds = None
        counter += 1
        gdf_dict['geometry'].append(wkt)
        gdf_dict['bounds'].append(bounds)
        if counter%100 == 0:
            print('\nCounter: {}'.format(counter))


Counter: 100

Counter: 200

Counter: 300

Counter: 400

Counter: 500

Counter: 600


In [ ]:
out = open('gdf_dict.pkl','wb')

In [ ]:
import pickle
pickle.dump(gdf_dict, out)
out.close()

In [ ]:
gdf_dict

In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
df = pd.DataFrame(gdf_dict)
df.head()

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
from shapely.geometry import Polygon
polygons =[]
for bound in df['bounds']:
  MINX, MINY, MAXX, MAXY = [bound[0],bound[1], bound[3],bound[4]] 
  polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))
  polygons.append(polygon)

In [ ]:
df['geometry'] = polygons


In [ ]:
df['geometry']

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:3857")

In [ ]:

gdf.to_crs(epsg=4326, inplace=True)

In [ ]:
geo_data = open('geo_data.pkl','wb')

In [ ]:
pickle.dump(gdf, geo_data)
geo_data.close()

In [ ]:
MINXx, MINYy, MAXXx, MAXYy = [-93.756155, 41.918015, -93.747334, 41.921429]
poly = Polygon(((MINXx, MINYy), (MINXx, MAXYy), (MAXXx, MAXYy), (MAXXx, MINYy), (MINXx, MINYy)))
grid = gpd.GeoDataFrame([poly], columns=["geometry"])
grid.set_crs(epsg=4326, inplace=True)

In [ ]:
gdf[gdf['geometry'].contains(grid['geometry'][0])]

In [ ]:
for idx, bound in enumerate(gdf['geometry']):
  if bound.contains(poly):
    print(idx)

In [ ]:
gdf.columns

In [ ]:
gdf.head(4)

In [ ]:
#gdf.to_csv("../data/meta_data.csv", index=False)

In [ ]:
gdf.shape

In [ ]:
# plot the shapefile
plt.rcParams['figure.figsize'] = 10,5
gdf.plot(color="white", edgecolor="#2e3131", figsize=(10, 5))
plt.title('USA: regions')

In [ ]:
gdf['area'] = gdf.area
# Returns a GeoSeries of points for each geometric centroid.
gdf['centroid'] = gdf.centroid
# Returns a GeoSeries of geometries representing the point or smallest rectangular
# polygon (with sides parallel to the coordinate axes) that contains each object.
gdf['envelope'] = gdf.envelope

In [ ]:
print(gdf.crs)

In [ ]:
gdf.crs = "EPSG:4326"

geo_data: geopandas data frame holding information of geographic geometry data: data frame containing values to be used for plotting the choropleth maps columns: the names of two columns — first column to be used as the key (‘NAME_2′ containing the names of the district in our case); second column are the values to be mapped (‘area’) key_on: Variable in geo_data to bind the data to. It must start with ‘feature’ fill_color: Can pass a hex code, color name, or one of these color palettes: ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’ fill_opacity: Opacity level, range 0–1 line_opacity: Geopath line color (boundary of the polygon) legend_name: Title for data legend

In [ ]:
usa_g = gdf.drop(['centroid', 'envelope'], axis=1)

In [ ]:
# Estimate centroids of the country
# --------------------
cent_x =gdf['geometry'].centroid.x.mean()
cent_y =gdf['geometry'].centroid.y.mean()

# Create a map object using Folium
# --------------------
map_usa = folium.Map(location=[cent_y, cent_x], zoom_start=7, tiles='OpenStreetMap')  

# Create the choropleth map
# --------------------
choro = folium.Choropleth(geo_data=usa_g,
                         name='choropleth',     
                          data=usa_g,
                          columns=['name', 'area'], 
#                           key_on='feature.NAME_2', 
                          fill_color='YlOrRd',
                          fill_opacity=0.6, 
                          line_opacity=0.8,
                          legend_name= "Area size across USA's subregion"
                          ).add_to(map_usa) 

# add labels to map
choro.geojson.add_child(folium.features.GeoJsonTooltip(fields=['name', 'area'],
                                                       aliases=['County', 'Mass'], 
                                                       labels = False))
folium.LayerControl().add_to(map_usa)


map_usa

In [ ]:
# Estimate centroids of the country
# --------------------
cent_x =gdf['geometry'].centroid.x.mean()
cent_y =gdf['geometry'].centroid.y.mean()

# Create a map object using Folium
# --------------------
map_usa = folium.Map(location=[cent_y, cent_x], zoom_start=7, tiles='OpenStreetMap')  

# Create the choropleth map
# --------------------
choro = folium.Choropleth(geo_data=usa_g,
                         name='choropleth',     
                          data=usa_g,
                          columns=['name', 'area'], 
#                           key_on='feature.NAME_2', 
                          fill_color='YlOrRd',
                          fill_opacity=0.6, 
                          line_opacity=0.8,
                          legend_name= "Area size across USA's subregion"
                          ).add_to(map_usa) 

# add labels to map
choro.geojson.add_child(folium.features.GeoJsonTooltip(fields=['name', 'area'],
                                                       aliases=['County', 'Mass'], 
                                                       labels = True))
folium.LayerControl().add_to(map_usa)


map_usa